In [1]:
import pandas as pd
import geopandas as gpd
from lets_plot import *

LetsPlot.setup_html()

In [2]:
states = pd.read_csv("../data/USA_States.csv")
states.head()

,State,Latitude,Longitude
0,Alabama,32.806671,-86.791130
1,Alaska,61.370716,-152.404419
2,Arizona,33.729759,-111.431221
3,Arkansas,34.969704,-92.373123
4,California,36.116203,-119.681564


In [3]:
LetsPlot.set(maptiles_zxy(url='https://maps.wikimedia.org/osm-intl/{z}/{x}/{y}@2x.png'))

In [4]:
ggplot(data=states) + geom_livemap(aes("Longitude", "Latitude"))

In [5]:
dept = pd.read_csv("../data/usgs_dept_by_state_2019.csv")

dept.head()

,State,State Debt,Unnamed: 2,Local Debt,Unnamed: 4,State and Local Debt,Unnamed: 6,Gross State Product,Unnamed: 8,Real State Growth %,Unnamed: 10,Population (million),
0,Alabama,10.7,g,26.1,g,36.8,g,228.0,g,1.1,g,4.9,e
1,Alaska,5.9,g,3.5,g,9.4,g,55.7,g,-0.2,g,0.7,e
2,Arizona,13.3,g,30.5,g,43.8,g,361.1,g,2.3,g,7.3,e
3,Arkansas,7.3,g,14.9,g,22.2,g,131.4,g,0.5,g,3.0,e
4,California,145.0,g,298.7,g,443.7,g,3080.6,g,2.0,g,39.5,e


In [6]:
dept_gg = dept \
        .drop([" ", "Unnamed: 2", "Unnamed: 4", "State and Local Debt", 
               "Unnamed: 6", "Unnamed: 8", "Real State Growth %", "Unnamed: 10", "Population (million)"], axis=1) \
        .set_index("State") \
        .stack() \
        .reset_index() \
        .rename(columns={"level_1": "Item", 0: "$B"})


dept_gg


,State,Item,$B
0,Alabama,State Debt,10.7
1,Alabama,Local Debt,26.1
2,Alabama,Gross State Product,228.0
3,Alaska,State Debt,5.9
4,Alaska,Local Debt,3.5
...,...,...,...
148,Wisconsin,Local Debt,22.4
149,Wisconsin,Gross State Product,348.1
150,Wyoming,State Debt,0.9
151,Wyoming,Local Debt,1.4


In [7]:
ggplot(dept_gg) \
+ geom_bar(aes("State", "$B", fill="Item"), stat="identity", position="dodge", sampling="none") \
+ scale_fill_manual(values=["#FE0968", "#FF75A6", "#007BCD"]) \
+ ggsize(1000, 500)

In [8]:
states_gdf = gpd.GeoDataFrame(states, geometry=gpd.points_from_xy(states.Longitude, states.Latitude))
states_gdf.head()

,State,Latitude,Longitude,geometry
0,Alabama,32.806671,-86.791130,POINT (-86.79113 32.80667)
1,Alaska,61.370716,-152.404419,POINT (-152.40442 61.37072)
2,Arizona,33.729759,-111.431221,POINT (-111.43122 33.72976)
3,Arkansas,34.969704,-92.373123,POINT (-92.37312 34.96970)
4,California,36.116203,-119.681564,POINT (-119.68156 36.11620)


In [9]:
ggplot(dept_gg) + geom_livemap(aes(sym_x="Item", sym_y="$B", fill="Item"), symbol='bar', size=30, map=states_gdf, map_join=["State", "State"])

